# Assignment about three factor and CAPM

#### Student ID:   M114020043
#### Name: 張光緯

CH 20.5  (20%)  
現在市場上有四種資產 A、B、C、Ｄ根據判斷寫出選取矩陣 P，看法向量Q，分別印出P, Q  
(a)資產 A 的收益率為 6%   
(b)資產 B 的收益率比資產 D 低 2%   
(c)資產 C 的收益率比 A  低 3%   

In [2]:
#下載並import套件
!pip install cvxopt
!pip install yfinance
!pip install pandas_datareader
import os
import numpy as np
import pandas_datareader.data as web
import datetime as dt
from scipy import linalg
import pandas as pd
import sys
import yfinance as yf
import portfolioopt as pfopt
import statsmodels.api as sm 

#確認路徑
sys.path.append('D:\Fintech\portfolioopt-master')
print("Current sys.path:", sys.path)


Current sys.path: ['C:\\Users\\user', 'D:\\Anaconda\\python310.zip', 'D:\\Anaconda\\DLLs', 'D:\\Anaconda\\lib', 'D:\\Anaconda', '', 'D:\\Anaconda\\lib\\site-packages', 'D:\\Anaconda\\lib\\site-packages\\win32', 'D:\\Anaconda\\lib\\site-packages\\win32\\lib', 'D:\\Anaconda\\lib\\site-packages\\Pythonwin', 'D:\\Fintech\\portfolioopt-master']


In [3]:
#繪製出pick matrix

pick1 = np.array([1,0,0,0])
pick2 = np.array([0,-1,0,1])
pick3 = np.array([1,0,-1,0])

#寫出看法向量

q1 = np.array([0.06])
q2 = np.array([0.02])
q3 = np.array([0.03])

P = np.array([pick1,pick2,pick3])

Q = np.array([q1,q2,q3])

#印出pick matrix和看法向量

print(P) 

print(Q)

[[ 1  0  0  0]
 [ 0 -1  0  1]
 [ 1  0 -1  0]]
[[0.06]
 [0.02]
 [0.03]]


CH20.6 (40%)  
假設上題中的資產 A、B、C、D分別為東元(1504)、中工(2515)、大魯閣(1432)、葡萄王 (1707)。 獲取這四支股票從 2014 年到 2016 年的股價資料， 並利用 BlackLitterman 模型建構資產組合及優化資產配置（利用上題P, Q）。

*用Markowitz模型優化資產配置時，因原套件portfolioopt不能用了，可以設os path 用教材提供的portfolioopt-master文件，也可以用cvxpy做

In [4]:
retData = pd.read_csv('retdata.csv',sep='\t',index_col='Date')

retData.index = pd.to_datetime(retData.index)
retData = retData*0.01
retData=retData.dropna()

#定義blacklitterman模型

def blacklitterman(returns,tau, P, Q):
    mu=returns.mean()
    sigma=returns.cov()
    pi1=mu
    ts = tau * sigma
    Omega = np.dot(np.dot(P,ts),P.T) * np.eye(Q.shape[0])
    middle = linalg.inv(np.dot(np.dot(P,ts),P.T) + Omega)  
    er = np.expand_dims(pi1,axis=0).T + np.dot(np.dot(np.dot(ts,P.T),middle),
                      (Q - np.expand_dims(np.dot(P,pi1.T),axis=1)))
    posteriorSigma = sigma + ts - np.dot(ts.dot(P.T).dot(middle).dot(P),ts)
    return [er, posteriorSigma]

#獲取東元(1504)、中工(2515)、大魯閣(1432)、葡萄王 (1707)四支股票在2014~2016的資料

a = yf.download('1504.TW', start='2014-01-01', end='2016-12-31')['Close']
aRet = ((a - a.shift(1)) / a.shift(1)).dropna()
aRet.name = '1504.TW'

b = yf.download('2515.TW', start='2014-01-01', end='2016-12-31')['Close']
bRet = ((b - b.shift(1)) / b.shift(1)).dropna()
bRet.name = '2515.TW'

c = yf.download('1432.TW', start='2014-01-01', end='2016-12-31')['Close']
cRet = ((c - c.shift(1)) / c.shift(1)).dropna()
cRet.name = '1432.TW'

d = yf.download('1707.TW', start='2014-01-01', end='2016-12-31')['Close']
dRet = ((d - d.shift(1)) / d.shift(1)).dropna()
dRet.name = '1707.TW'

#將四個數據框已列的方向合併

stock=pd.concat([aRet,bRet,cRet,dRet],axis=1)
stock=stock.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [7]:
# 修正後驗收益

res=blacklitterman(stock,0.1, P, Q)
p_mean=pd.DataFrame(res[0],index=stock.columns,columns=['posterior_mean'])
p_mean

,posterior_mean
1504.TW,0.030386
2515.TW,0.002842
1432.TW,0.001690
1707.TW,0.014043


In [8]:
p_cov=res[1]
p_cov

,1504.TW,2515.TW,1432.TW,1707.TW
1504.TW,0.000335,0.000059,0.000025,0.000076
2515.TW,0.000059,0.000190,0.000043,0.000070
1432.TW,0.000025,0.000043,0.000389,0.000093
1707.TW,0.000076,0.000070,0.000093,0.000492


In [9]:
target_ret2 = 0.0006
cov_mat2 = res[1]
avg_rets2 = pd.Series(res[0][:,0],index=stock.columns)
weights2 = pfopt.markowitz_portfolio(cov_mat2, avg_rets2, target_ret2,
  allow_short=False)
weights2

1504.TW    0.229911
2515.TW    0.460656
1432.TW    0.211506
1707.TW    0.097927
dtype: float64

CH22.6 (40%)   
讀取宏達電 (2498) 的 2014 年資料以及三因子資料   
(a)利用 2014 年度的數據建立 CAPM 模型   
(b)利用 2014 年度的數據建立三因子模型  
(c)分別使用建構的 CAPM 模型與三因子模型估計 factors2.csv 宏達電 2015 年 1 月的收益率，分別比較兩者的結果

In [10]:
#讀取宏達電(2498)的資料

stock2498 = pd.read_csv('D:/Fintech/data/2498.csv')
stock2498['Date'] = pd.to_datetime(stock2498['Date'])
stock2498.head()

,Date,Date.1,Close,Open,High,Low,Volume,ROI
0,2014-01-02,2014/1/2,137.58,140.56,141.06,137.09,7964,-1.7730
1,2014-01-03,2014/1/3,137.58,137.58,140.07,137.09,6599,0.0000
2,2014-01-06,2014/1/6,132.62,137.58,138.58,132.12,14005,-3.6101
3,2014-01-07,2014/1/7,130.13,131.62,132.62,128.64,13705,-1.8727
4,2014-01-08,2014/1/8,129.14,130.13,131.62,129.14,7623,-0.7634


In [11]:
#下載三因子模型

stockfactor = pd.read_csv('D:/Fintech/data/factors.csv')
stockfactor['YY/MM/DD'] = pd.to_datetime(stockfactor['YY/MM/DD'])
stockfactor.head()

,Security Name,CoName,YY/MM/DD,Market Risk Premium,Size Factor (3 Factor),Book to Market Factor,Risk-free Interest Rate
0,Z8888,TSEC+GreTai Non-Banking index,2014-01-02,0.0145,0.7197,0.3039,1.355
1,Z8888,TSEC+GreTai Non-Banking index,2014-01-03,-0.5858,0.6868,1.0533,1.355
2,Z8888,TSEC+GreTai Non-Banking index,2014-01-06,-0.4534,0.5783,0.4498,1.355
3,Z8888,TSEC+GreTai Non-Banking index,2014-01-07,0.1950,0.4554,0.7056,1.355
4,Z8888,TSEC+GreTai Non-Banking index,2014-01-08,0.6204,-0.0295,0.2259,1.355


In [12]:
#將三因子和宏達電資料合併

mergedata = pd.concat([stock2498.set_index('Date'), stockfactor.set_index('YY/MM/DD')], axis=1, join='inner')
mergedata

,Date.1,Close,Open,High,Low,Volume,ROI,Security Name,CoName,Market Risk Premium,Size Factor (3 Factor),Book to Market Factor,Risk-free Interest Rate
2014-01-02,2014/1/2,137.58,140.56,141.06,137.09,7964,-1.7730,Z8888,TSEC+GreTai Non-Banking index,0.0145,0.7197,0.3039,1.355
2014-01-03,2014/1/3,137.58,137.58,140.07,137.09,6599,0.0000,Z8888,TSEC+GreTai Non-Banking index,-0.5858,0.6868,1.0533,1.355
2014-01-06,2014/1/6,132.62,137.58,138.58,132.12,14005,-3.6101,Z8888,TSEC+GreTai Non-Banking index,-0.4534,0.5783,0.4498,1.355
2014-01-07,2014/1/7,130.13,131.62,132.62,128.64,13705,-1.8727,Z8888,TSEC+GreTai Non-Banking index,0.1950,0.4554,0.7056,1.355
2014-01-08,2014/1/8,129.14,130.13,131.62,129.14,7623,-0.7634,Z8888,TSEC+GreTai Non-Banking index,0.6204,-0.0295,0.2259,1.355
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-26,2014/12/26,136.59,135.10,137.09,135.10,7532,1.8519,Z8888,TSEC+GreTai Non-Banking index,0.6399,-0.2479,0.0360,1.355
2014-12-27,2014/12/27,136.59,136.59,137.09,135.60,2560,0.0000,Z8888,TSEC+GreTai Non-Banking index,0.1055,0.5418,0.0852,1.355
2014-12-29,2014/12/29,136.59,136.59,137.58,135.60,3866,0.0000,Z8888,TSEC+GreTai Non-Banking index,0.7341,-0.2407,-0.4195,1.355
2014-12-30,2014/12/30,139.57,137.09,140.07,136.09,10812,2.1818,Z8888,TSEC+GreTai Non-Banking index,-0.1990,0.1720,-0.2125,1.355


In [13]:
#(a)利用 2014 年度的數據建立 CAPM 模型

model=sm.OLS(mergedata['ROI'],sm.add_constant(mergedata['Market Risk Premium'])).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ROI   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     32.30
Date:                Thu, 28 Dec 2023   Prob (F-statistic):           3.72e-08
Time:                        11:30:18   Log-Likelihood:                -516.02
No. Observations:                 248   AIC:                             1036.
Df Residuals:                     246   BIC:                             1043.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.0235      0.124     -0.190      0.850      -0.267       0.220
Market Risk Premium     1.0046      0.177      5.683      0.000       0.656       1.353
==============================================================================
Omnibus:                       14.984   Durbin-Watson:                   2.192
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               28.565
Skew:                           0.300   Prob(JB):                     6.27e-07
Kurtosis:                       4.551   Cond. No.                         1.44
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
#(b)利用 2014 年度的數據建立三因子模型

regThrFac=sm.OLS(mergedata['ROI'],sm.add_constant(mergedata.iloc[:,9:12])).fit()
regThrFac.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ROI   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     13.57
Date:                Thu, 28 Dec 2023   Prob (F-statistic):           3.24e-08
Time:                        11:30:49   Log-Likelihood:                -512.19
No. Observations:                 248   AIC:                             1032.
Df Residuals:                     244   BIC:                             1046.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0.0282      0.123     -0.230      0.819      -0.270       0.213
Market Risk Premium        1.0097      0.181      5.587      0.000       0.654       1.366
Size Factor (3 Factor)     0.0253      0.257      0.099      0.921      -0.480       0.531
Book to Market Factor      0.8823      0.335      2.631      0.009       0.222       1.543
==============================================================================
Omnibus:                       13.947   Durbin-Watson:                   2.109
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               24.772
Skew:                           0.301   Prob(JB):                     4.18e-06
Kurtosis:                       4.427   Cond. No.                         2.88
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
#(c)分別使用建構的 CAPM 模型與三因子模型估計 factors2.csv 宏達電 2015 年 1 月的收益率，分別比較兩者的結果

#讀取factor2的資料來進行預測

stock2015 = pd.read_csv('D:/Fintech/data/factors2.csv',sep='\t')

# 將日期欄位轉換成日期型別

stock2015['YY/MM/DD'] = pd.to_datetime(stock2015['YY/MM/DD'])

# 選擇一月份的資料

jandata = stock2015[(stock2015['YY/MM/DD'].dt.month == 1) & (stock2015['YY/MM/DD'].dt.year == 2015) ]

predictcapm = model.predict(sm.add_constant(jandata['Market Risk Premium']),
   transform=False)

predictThrFac = regThrFac.predict(sm.add_constant(jandata.loc[:, ['Market Risk Premium', 'Size Factor (3 Factor)', 'Book to Market Factor']]),
  transform=False)

print("\n")
print('CAPM模型下預測的收益率 : ',predictcapm.mean())
print("\n")
print('三因子模型下預測的收益率 : ',predictThrFac.mean())
print("\n")



CAPM模型下預測的收益率 :  0.0048302075157619865


三因子模型下預測的收益率 :  0.06956786126957001


從預測的結果來看，三因子模型預測的收益率是比CAPM模型還要高的(兩者有蠻大的差異)，學生推測可能的原因應該是CAPM模型是根據單一的風險因子去進

行預測，如講義所說的，可能單一風險因子的無法充分解釋個別風險資產的收益率，而三因子模型作為經由CAPM改良後的模型，考慮了市場規模以外的因子也

改良了CAPM的局限和不足，考慮的規模可能較大，解釋也更全面，較有說服力，所以造成預測的差異。